<img style="float: left;width: 300px;" src="MongoPython.png" alt="PyMongo"/>

# Getting started with Python + Mongo: Pymongo
<br><br>
## Mongo installation Guides
- [Ubuntu](https://docs.mongodb.com/manual/tutorial/install-mongodb-on-ubuntu/)
- [OSX](https://docs.mongodb.com/manual/tutorial/install-mongodb-on-os-x/)
<br><br>
## Pymongo
- Install with: `python -m pip install pymongo`
- [Documentation](https://api.mongodb.com/python/current/)
<br><br>
## Notes
This demo notebook asumes you have installed MongoDB locally and in `mongod` is running on it's default port `27017`


In [71]:
from pymongo import MongoClient
import datetime
print('Using pymongo version: {0}'.format(pymongo.__version__))

Using pymongo version: 3.7.2


In [72]:
#connect to mongod running in localhost 
client = MongoClient('localhost', 27017)

In [74]:
#drop database
client.drop_database('test_database')

In [75]:
# list databases
# client.database_names()

/home/cm/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:2: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  


['admin', 'config', 'local']

In [18]:
#create database
db = client.test_database

In [26]:
post = {"author": "Mike", 
        "text": "My first blog post!",
        "tags": ["mongodb", "python", "pymongo"],
        "date": datetime.datetime.utcnow()}

In [27]:
# insert a document into a collection
posts = db.posts
post_id = posts.insert_one(post).inserted_id
post_id

ObjectId('5beb06bfcbc0e205661e932d')

In [28]:
# validate new dababase has been created
client.database_names()

/home/cm/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  """Entry point for launching an IPython kernel.


['admin', 'config', 'local', 'test_database']

In [29]:
# find new collection in database
db.collection_names(include_system_collections=False)

/home/cm/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['posts']

In [31]:
#query collection for any match
posts.find_one()

{'_id': ObjectId('5beb06bfcbc0e205661e932d'),
 'author': 'Mike',
 'date': datetime.datetime(2018, 11, 13, 17, 14, 17, 432000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}

In [32]:
# query with target
posts.find_one({"author": "Mike"})

{'_id': ObjectId('5beb06bfcbc0e205661e932d'),
 'author': 'Mike',
 'date': datetime.datetime(2018, 11, 13, 17, 14, 17, 432000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}

In [33]:
# query by object id
posts.find_one({"_id": post_id})

{'_id': ObjectId('5beb06bfcbc0e205661e932d'),
 'author': 'Mike',
 'date': datetime.datetime(2018, 11, 13, 17, 14, 17, 432000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}

In [35]:
# a string representation of object id is not the same and does not return data
post_id_as_str = str(post_id)
print(post_id_as_str)
posts.find_one({"_id": post_id_as_str})

5beb06bfcbc0e205661e932d


In [43]:
from bson.objectid import ObjectId
# Convert from string to ObjectId:
posts.find_one({'_id': ObjectId(post_id_as_str)})

{'_id': ObjectId('5beb06bfcbc0e205661e932d'),
 'author': 'Mike',
 'date': datetime.datetime(2018, 11, 13, 17, 14, 17, 432000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}

In [44]:
#more interesting queries
new_posts = [{"author": "Mike",
              "text": "Another post!",
              "tags": ["bulk", "insert"],
              "date": datetime.datetime(2009, 11, 12, 11, 14)},
             {"author": "Eliot",
              "title": "MongoDB is fun",
              "text": "and pretty easy too!",
              "date": datetime.datetime(2009, 11, 10, 10, 45)}]

In [45]:
result = posts.insert_many(new_posts)
result.inserted_ids

In [50]:
for post in posts.find():
    print(post)

{'author': 'Mike', '_id': ObjectId('5beb06bfcbc0e205661e932d'), 'date': datetime.datetime(2018, 11, 13, 17, 14, 17, 432000), 'text': 'My first blog post!', 'tags': ['mongodb', 'python', 'pymongo']}
{'author': 'Mike', '_id': ObjectId('5beb0a42cbc0e205661e932e'), 'date': datetime.datetime(2009, 11, 12, 11, 14), 'text': 'Another post!', 'tags': ['bulk', 'insert']}
{'author': 'Eliot', '_id': ObjectId('5beb0a42cbc0e205661e932f'), 'date': datetime.datetime(2009, 11, 10, 10, 45), 'title': 'MongoDB is fun', 'text': 'and pretty easy too!'}


In [61]:
for post in posts.find({"author": "Mike"}):
    print(post)

{'author': 'Mike', '_id': ObjectId('5beb06bfcbc0e205661e932d'), 'date': datetime.datetime(2018, 11, 13, 17, 14, 17, 432000), 'text': 'My first blog post!', 'tags': ['mongodb', 'python', 'pymongo']}
{'author': 'Mike', '_id': ObjectId('5beb0a42cbc0e205661e932e'), 'date': datetime.datetime(2009, 11, 12, 11, 14), 'text': 'Another post!', 'tags': ['bulk', 'insert']}


In [66]:
posts.count_documents({"text": "and pretty easy too!"})

1

In [68]:
d = datetime.datetime(2009, 11, 12, 12)
for post in posts.find({"date": {"$lt": d}}).sort("author"):
    print(post)

{'author': 'Eliot', '_id': ObjectId('5beb0a42cbc0e205661e932f'), 'date': datetime.datetime(2009, 11, 10, 10, 45), 'title': 'MongoDB is fun', 'text': 'and pretty easy too!'}
{'author': 'Mike', '_id': ObjectId('5beb0a42cbc0e205661e932e'), 'date': datetime.datetime(2009, 11, 12, 11, 14), 'text': 'Another post!', 'tags': ['bulk', 'insert']}


In [70]:
#close the connection
client.close()